# scGenePT Perturbation Prediction Tutorial 

<a id = 'introduction'></a>
Last Updated: [11/14/2024] This notebook offers examples of how to use trained scGenePT models in inference mode for perturbation prediction. It uses models fine-tuned on the **Norman [1] dataset** and offers examples of predicting post-perturbation expression responses for **single gene perturbations**: POU3F2, CDKN1B and **gene combinations**: SAMD1+ZBTB1. Models have not been evaluated on multiple gene combination perturbation responses, so behavior is unknown. 

**Model** : scGenePT is a collection of single-cell models for perturbation prediction. It leverages the [scGPT](https://github.com/bowang-lab/scGPT) [2] foundation model for scRNAseq data by injecting language embeddings at the gene level into the model architecture. The language gene embeddings are obtained by embedding gene level information from different knowledge sources using LLMs. The knowledge sources used include NCBI gene descriptions, UniProt protein Summaries for protein coding genes - as inspired by the [genePT](https://github.com/yiqunchen/GenePT) [3] approach - and GO (Gene Ontology) Gene Molecular Annotations, across three different axes: Molecular Function, Biological Process and Cellular Component. The model variations available are:
- **scGenePT_NCBI** = scGPT + NCBI Gene Card Summaries
- **scGenePT_NCBI+UniProt** = scGPT + NCBI Gene Card Summaries + UniProt Protein Summaries
- **scGenePT_GO−F** = scGPT + GO Molecular Functions Annotations
- **scGenePT_GO−C** = scGPT + GO Cellular Components Annotations
- **scGenePT_GO−P** = scGPT + GO Biological Processes Annotations
- **scGenePT+GO−all** = scGPT + GO_F + GO_C + GO_P

**Dataset**: The Norman Dataset [1] is a CRISPR perturb-seq dataset containing single and two-gene perturbations. We use a processed version of the dataset that contains 105 single and 131 two-gene combinations perturbations coming from 91k observations. Cells in the dataset are log-normalized and filtered to the top 5000 highly variable genes. Models in ths notebook are trained on the train split of the dataset. We offer examples of:
- performing inference on the test split of the Norman dataset
- performing inference on a random control sample and on a new anndata file

**The notebook is structured and offers the following examples**:
1. [Perturbation prediction](#perturbation_prediction)
    - [Load the Norman dataset](#load_dataset)
    - [Load a trained scGenePT model](#load_trained_scgenept_model)
2. [Plotting the top 20 Differentially Expressed Genes post-perturbation](#plot_top_20_de)
    - [Predicting perturbation response for perturbing the POU3F2 gene. Comparison of scGPT with scGenePT_GO−C](#example_POU3F2+ctrl)
    - [Predicting perturbation response for perturbing the CDKN1B gene. Comparison of scGPT with scGenePT_GO−C](#example_CDKN1B+ctrl)
    - [Predicting perturbation response for perturbing the SAMD1+ZBTB1 gene combination. Comparison of scGPT with scGenePT_NCBI+UniProt](#example_SAMD1+ZBTB1) 
3. [Perturbation prediction on NumPy arrays holding control samples](#prediction_on_rnd_ctrl)
4. [Perturbation prediction on AnnData files](#prediction_on_new_anndata_file)

<!-- **The full list of resources available is**:
1. [bioRxiv Preprint](https://www.biorxiv.org/content/10.1101/2024.10.23.619972v1)
2. [scGenePT Trained Model Weights](https://drive.google.com/drive/u/0/folders/12h1hL3cJF3W0VG92JqGJ1-4R-2nDXbzc)
3. [Gene Embeddings, computed using LLMs](https://drive.google.com/drive/folders/191d8uXaoUNvvZ8DZHzlR1O6BK9vLnqqy?usp=drive_link) <br>
• NCBI Gene Card summaries, NCBI Gene Card Summaries + UniProt Protein Summaries <br>
• GO-C Cellular Components Gene Annotations, GO-F Molecular Function Gene Annotations, GO-P Biological Process Gene Annotations, GO-all (GO-C + GO-F + GO-P) -->

## Setup

**Prerequisites and hardware requirements**

1. The notebook must run inside a virtual env containing the needed packages. If you already created one, then you can skip this step. If you haven't, you must create one through. <br>

First, you'll need to clone the repo through: <br>

```
git clone git@github.com:czi-ai/scGenePT.git
```


Then you need to create a virtual environment containing the model packaging dependencies. The `requirements.txt` file is under the repo.

```python
conda create -y --name scgenept python=3.10
source activate scgenept
pip install -r requirements.txt
pip install flash-attn --no-build-isolation
pip install scgpt "flash-attn<1.0.5"
```

2. Once you have a venv created, activate it and allow the jupyter notebook to use it as a kernel by running the following command from the terminal. If jupyter is not installed, it should be:
```python
source activate scgenept
pip install ipykernel
pip install jupyter
python -m ipykernel install --user --name=scgenept
```

3. Open the jupyter notebook. You should now be able to see and select a new kernel: scgenept. The notebook needs to be ran inside this kernel in order for all packages to be properly installed.

**Model and Data**

Please follow these steps on the [GitHub README](https://github.com/czi-ai/scGenePT/blob/main/README.md): 
- **Download scGPT Pretrained Model** - the scGPT model needs to be under `models/pretrained/scgpt` 
- **Download Gene Embeddings** - the gene embeddings files should be under `models/gene_embeddings`. 
- **Download Trained scGenePT models from the scGenePT Model Zoo** - they should be under `models/finetuned` <br>
Note that you can download all finetuned models from [this link](https://drive.google.com/drive/folders/1U9PodoV7A-Dkk-GemmLB_AzmkgE-owp_?usp=drive_link)


**The expected folder structure should be**:

- models
    - pretrained/scgpt
    - gene_embeddings
    - finetuned
- tutorials
    - scgenept_tutorial.ipynb

In [ ]:
import sys
sys.path.insert(1, '../')

from train import load_dataloader
from utils.data_loading import *
from models.scGenePT import *
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from gears.inference import evaluate, compute_metrics, deeper_analysis, non_dropout_analysis

<a id='perturbation prediction'></a>
## 1. Perturbation Prediction using a trained scGenePT model

<a id='load_dataset'></a>
### Load dataset
We start by loading the Norman dataset. The dataloaders use a pre-processed version of Norman from [GEARS](https://github.com/snap-stanford/GEARS) [3]. Models in this notebook have been finetuned on the train split of this dataset. We also showcase how to perform inference on the test split of this dataset. 

In [ ]:
# Has been tested with 'norman' and 'adamson'
dataset_name = 'norman' 
batch_size = 64
eval_batch_size = 64

pert_data = load_dataloader(dataset_name, batch_size, eval_batch_size, split = 'simulation')
train_loader = pert_data.dataloader['train_loader']
val_loader = pert_data.dataloader['val_loader']
test_loader = pert_data.dataloader['test_loader']

<a id='dataset_exploration'></a>
### Dataset Exploration

We can examine the data inside the dataloaders. The `pert_data.adata` is an AnnData object containing gene expression counts and additional information.

In [ ]:
pert_data.adata

The gene expression counts are stored in `pert_adata.adata.X`, where each entry (i, j) entry corresponds to the gene expression counts of gene j in row i

In [ ]:
pert_data.adata.X # 902105 cells, 5045 genes

The `pert_data.adata.obs` file contains information about the different perturbation conditions applied to the data stored in .X. For instance, below we can see examples of 1-gene perturbations: *TSC22D1+ctrl, MAML2+ctrl*, etc, examples of 2-gene perturbations: *KLF1+MAP2K6, CEBPE+RUNX1T1*, and examples of control samples. Each of the conditions corresponds to gene expression counts stored in .X.

In [ ]:
pert_data.adata.obs # perturbation conditions applied to the 902105 cells

The `pert_data.adata.var` gives us the set of genes:

In [ ]:
print(f'There are {len(pert_data.adata.var)} number of genes present in the dataset.')
pert_data.adata.var.head() # the names of the 5045 genes

<a id='load_trained_scgenept'></a>
### Load trained scGenePT models

In this section, we show examples of loading multiple scGenePT models for prediction in order to compare them. All the models under the scGenePT Model Zoo can be found under [GitHub repo](https://github.com/czi-ai/scGenePT/) and the trained model weights under [this GDrive link](https://drive.google.com/drive/folders/1U9PodoV7A-Dkk-GemmLB_AzmkgE-owp_?usp=drive_link)<br>

Here, we load the following:
- **scGPT** = scGPT model | [Download Link](https://drive.google.com/file/d/1OzTVmj24k_QrLAwKkwv8Q6SNW4yDcW00/view?usp=drive_link)
- **scGenePT_NCBI+UniProt** = scGPT + NCBI Gene Card Summaries + UniProt Protein Summaries | [Download Link](https://drive.google.com/file/d/17tsBhTkuEw1uwTSvsK9Hskq3px_3MIQU/view?usp=drive_link)
- **scGenePT_GO−C** = scGPT + GO Cellular Components Annotations | [Download Link](https://drive.google.com/file/d/17tsBhTkuEw1uwTSvsK9Hskq3px_3MIQU/view?usp=drive_link)

Note that models are assumed to be under the `models/finetuned/{dataset_name}/{model_type}/best_model_seed_42.pt` folder


**Once the Google Drive files are public, this section should use gdown to automatically download the data under the right folder structure**

In [ ]:
def load_trained_scgenept_model(pert_data, model_type, models_dir, model_location, device, verbose = False):
    embs_to_include = get_embs_to_include(model_type)
    vocab_file = models_dir + 'pretrained/scgpt/vocab.json'
    vocab, gene_ids, dataset_genes, gene2idx = match_genes_to_scgpt_vocab(vocab_file, pert_data, True, SPECIAL_TOKENS)
    ntokens = len(vocab)  # size of vocabulary
    genept_embs, genept_emb_type, genept_emb_dim, found_genes_genept = initialize_genept_embeddings(embs_to_include, dataset_genes, vocab, model_type, models_dir)
    go_embs_to_include, go_emb_type, go_emb_dim, found_genes_go = initialize_go_embeddings(embs_to_include, dataset_genes, vocab, model_type, models_dir)
    
    model = scGenePT(
        ntoken=ntokens,
        d_model=EMBSIZE,
        nhead=NHEAD,
        d_hid=D_HID,
        nlayers=NLAYERS,
        nlayers_cls=N_LAYERS_CLS,
        n_cls=N_CLS,
        vocab=vocab,
        n_perturbagens=2,
        dropout=0.2,
        pad_token=PAD_TOKEN,
        pad_value=PAD_VALUE,
        pert_pad_id=PERT_PAD_ID,
        use_fast_transformer=True,
        embs_to_include = embs_to_include,
        genept_embs = genept_embs, 
        genept_emb_type = genept_emb_type, 
        genept_emb_size = genept_emb_dim,
        go_embs_to_include = go_embs_to_include,
        go_emb_type = go_emb_type,
        go_emb_size = go_emb_dim
    )
    
    model.load_state_dict(torch.load(model_location, weights_only=True))
    if verbose:
        print(model)
    model.to(device)
    return model, gene_ids

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device {device}')

# Mapping from model name to the name of the model file model weights are saved
# Note that if you want to experiment with only one or a couple of models only, not all models need to be downloaded
model_name2model_variation = {'scgpt' : 'best_model_seed_42.pt', 
                    'scgenept_ncbi+uniprot_gpt' : 'best_model_gpt3.5_ada_rnd_seed_42.pt',
                    'scgenept_go_c_gpt_concat' : 'best_model_gpt3.5_ada_rnd_seed_42_concat.pt'}

# To load all the models, uncomment this code and download the specific models from the GDrive link above
# model_name2model_variation = {'scgpt' : 'best_model_seed_42.pt', 
#                     'scgenept_ncbi_gpt' : 'best_model_gpt3.5_ada_rnd_seed_42.pt',
#                     'scgenept_ncbi+uniprot_gpt' : 'best_model_gpt3.5_ada_rnd_seed_42.pt',
#                     'scgenept_go_c_gpt_concat' : 'best_model_gpt3.5_ada_rnd_seed_42_concat.pt',
#                     'scgenept_go_f_gpt_concat' : 'best_model_gpt3.5_ada_rnd_seed_42_concat.pt',
#                     'scgenept_go_p_gpt_concat' : 'best_model_gpt3.5_ada_rnd_seed_42_concat.pt',
#                     'scgenept_go_all_gpt_concat' : 'best_model_gpt3.5_ada_rnd_seed_42_concat.pt'}

# Names of the scGenePT models to load. Note that these have to match the keys in the model_name2model_variation dict
models = ['scgpt',  'scgenept_go_c_gpt_concat', 'scgenept_ncbi+uniprot_gpt']
# models = ['scgpt',  'scgenept_go_c_gpt_concat', 'scgenept_go_p_gpt_concat', 'scgenept_go_f_gpt_concat', 'scgenept_go_all_gpt_concat', 'scgenept_ncbi_gpt', 'scgenept_ncbi+uniprot_gpt']
trained_models = {}

# Location of where the pretrained scGPT model and gene embeddings are located
pretrained_scgpt_model_dir = '../models/'

for model_name in models:
    print(f"Now loading a {model_name} model ... ")
    print('=' * 30)
    model_filename = model_name2model_variation[model_name]
    if model_name != 'scgpt':
        model_prefix = ''.join(model_name.split('_gpt')[:-1]) 
    else:
        model_prefix = model_name
    model_location =  f'../models/finetuned/{model_prefix}/{dataset_name}/{model_filename}'
    model, gene_ids =  load_trained_scgenept_model(pert_data, model_name, pretrained_scgpt_model_dir, model_location, device)
    print('Done!\n')
    trained_models[model_name] = model

<a id='plot_top_20_de'></a>
## 2. Plot the Top Differentially Expressed Genes post-perturbation
We can use the trained models to perform inference. Below, we offer an example of how to do this on the test split of the Norman dataset - this is novel data the model has not been trained on. The same workflow/analyses can be applied to other data to get similar insights. For instance, in the following sections we offer examples of performing inference  on completely new data, either in numpy or anndata form.

We explore the following scenarios and showcase how to plot the top differentially expressed genes post-perturbation for: <br>
• **single-gene perturbation**: predicting the effects of perturbing the POU3F2 gene and the CDKN1B gene individually. We offer examples of comparing scGPT with scGenePT_GO−C model predictions <br>
• **two-gene perturbation**: predicting the effect of perturbing the SAMD1+ZBTB1 genes simultaneously. We offer an example of comparing scGPT and scGenePT_NCBI+UniProt model predictions

In [ ]:
def plot_perturbation(
    model: nn.Module, query: str, model_type, color, marker, title, save_file: str = None, amp = True, pool_size: int = None
):
    import seaborn as sns
    import numpy as np
    import matplotlib.pyplot as plt

    sns.set_theme(style="ticks", rc={"axes.facecolor": (0, 0, 0, 0)}, font_scale=1.5)

    adata = pert_data.adata
    adata_ctrl = adata[adata.obs['condition'] == 'ctrl']
    gene_names = pert_data.gene_names.to_list()
    
    gene2idx = pert_data.node_map
    cond2name = dict(adata.obs[["condition", "condition_name"]].values)
    gene_raw2id = dict(zip(adata.var.index.values, adata.var.gene_name.values))

    de_idx = [
        gene2idx[gene_raw2id[i]]
        for i in adata.uns["top_non_zero_de_20"][cond2name[query]]
    ]
    genes = [
        gene_raw2id[i] for i in adata.uns["top_non_zero_de_20"][cond2name[query]]
    ]
    truth = adata[adata.obs.condition == query].X.toarray()[:, de_idx]
    pred = model.pred_perturb_from_ctrl(adata_ctrl, query, gene_names, device, gene_ids, amp, pool_size).squeeze()[de_idx]
    ctrl_means = adata[adata.obs["condition"] == "ctrl"].to_df().mean()[de_idx].values

    pred = pred - ctrl_means
    truth = truth - ctrl_means

    plt.figure(figsize=[16.5, 4.5])
    plt.title(title + '\n' + query)
    plt.boxplot(truth, showfliers=False, medianprops=dict(linewidth=0))

    for i in range(pred.shape[0]):
        _ = plt.scatter(i + 1, pred[i], color=color, marker=marker)

    plt.axhline(0, linestyle="dashed", color="green")

    ax = plt.gca()
    ax.xaxis.set_ticklabels(genes, rotation=90)

    plt.ylabel("Change in Gene Expression over Control", labelpad=10)
    plt.tick_params(axis="x", which="major", pad=5)
    plt.tick_params(axis="y", which="major", pad=5)
    sns.despine()

Before we perform predictions, we can explore the test split of the Norman dataset in more detail. We are interested in this split, because this is data that the model has not seen during training. So performance on this split gives us a sense of model performance on unseen data. The `pert_data.subgroup` object contains information about the set of perturbations present in each of the train/val/test splits, and each can be explored in more detail. For example, the perturbation types in the test set are split into the following categories: 
- **unseen_single**: single gene perturbations that have not been seen during training
- **combo_seen0**: two-gene perturbations, none of the genes has been seen perturbed (individually or as part of a combination) during training 
- **combo_seen1**: two-gene perturbations, one of the genes has been seen perturbed (individually or as part of a combination) during training
- **combo_seen2**: two-gene perturbations, both genes has been seen perturbed (individually or as part of a combination) during training

In [ ]:
pert_data.subgroup['test_subgroup'].keys()

unseen_single = pert_data.subgroup['test_subgroup']['unseen_single']
combo_seen0 = pert_data.subgroup['test_subgroup']['combo_seen0']
combo_seen1 = pert_data.subgroup['test_subgroup']['combo_seen1']
combo_seen2 = pert_data.subgroup['test_subgroup']['combo_seen2']

print(f'unseen_single: {len(unseen_single)}, combo_seen0: {len(combo_seen0)}, combo_seen1: {len(combo_seen1)}, combo_seen2: {len(combo_seen2)}')

Examples of combinations of genes where none of the genes has been seen during training:

In [ ]:
combo_seen0

<a id='example_POU3F2+ctrl'></a>
### Single-Gene Perturbation: Predicting perturbation response for perturbing the POU3F2 gene

Then we can start exploring prediction on some of these perturbation categories. In the example below, we compare the perturbation prediction responses of scGPT and scGenePT_GO-C - scGPT + GO Cellular Components Annotations by perturbing the POU2F2 gene. Note that this gene has not been seen perturbed during training by the model. We do this by sampling n = 300 random controls from the training dataset and taking the mean of the average prediction for each control. According to NCBI Gene Card https://www.ncbi.nlm.nih.gov/gene/5454, overexpression of the protein encoded by POU3F2 is associated with an increase in the proliferation of melanoma cells. We can see that the genes FABP5, HSP90AB1, PRDX1, NPM1, TMSB10, PTMA are all better predicted as having a negative fold change over control by scGenePTGO−C, compared to scGPT which predicts a non-significant effect. 

In [ ]:
pert = 'POU3F2+ctrl'
colors = ['fuchsia', 'blue', 'red']
marker_types = [ 'o','s', 'x']
models_to_compare = ['scgpt', 'scgenept_go_c_gpt_concat']

for model, color, marker_type, title  in zip(models_to_compare, colors, marker_types, ['scGPT', 'scGenePT_GO-C']):
    plot_perturbation(trained_models[model], pert, model, color, marker_type, title, amp = True, pool_size=300)

We can inspect the GO Gene Annotations that were used for this gene during training. If you don't already have the GO annotations downloaded, they can be retrieved from [here](https://drive.google.com/drive/folders/1NrRfw5-N0GAZSOQ5EBI5xRL7dj3doHZP?usp=drive_link)

In [ ]:
import pandas as pd
gene_annotations_dir = '../models/gene_embeddings/gene_annotations/'

#GO-C Cellular Component Annotations
GO_C_annotations_df = pd.read_csv(gene_annotations_dir + 'gene_ontology_C.csv')
#GO-P Biological Process Annotations
GO_P_annotations_df = pd.read_csv(gene_annotations_dir + 'gene_ontology_P.csv')
#GO-F Molecular Function Annotations
GO_F_annotations_df = pd.read_csv(gene_annotations_dir + 'gene_ontology_F.csv')

For example, we can look at the GO-C Cellular Components annotations for the POU3F2 gene:

In [ ]:
GO_C_annotations_df[GO_C_annotations_df['gene'] == 'POU3F2']

By adding the GO Cellular Component annotations, the model learns that this gene is localized mostly in: nucleoplasm, chromatin and transcription regulator complex. Localization of gene products in the cell plays an important role in their biological function, e.g. protein-protein interaction; regulation of gene expression, transportation of protein. This tells us that subcellular localization is helpful in being able to predict effects of perturbation of this gene.

<a id='#example_CDKN1B+ctrl'></a>
### Single-Gene Perturbation: Predicting perturbation response for perturbing the CDKN1B gene
We can do this analysis for other genes as well. For example, the CDKN1B gene is another gene that has not been seen perturbed during training. According to this gene’s NCBI Gene Card https://www.ncbi.nlm.nih.gov/gene/1027, mutations in this gene are associated with multiple enodcrine neoplasia type IV. We can see that scGenePTGO−C predicts HSP90AA1, PTMA, RANBP1, CKS1B, PRDX1, PHF19 and NME1 as correctly down-regulated, as opposed to scGPT which predicts either neutral effect or positive fold change. Similarly, we speculate that the model learns to incorporate cellular location information to better predict gene expression change in response to genetic perturbation.

In [ ]:
pert = 'CDKN1B+ctrl'
colors = ['fuchsia', 'blue']
marker_types = [ 'o','s']
models_to_compare = ['scgpt', 'scgenept_go_c_gpt_concat']

for model, color, marker_type, title  in zip(models_to_compare, colors, marker_types, ['scGPT', 'scGenePT_GO-C']):
    plot_perturbation(trained_models[model], pert, model, color, marker_type, title, amp = True, pool_size=300)

We can see that the subcellular localization annotations that the model sees during training for the CDK1NB gene are:

In [ ]:
GO_C_annotations_df[GO_C_annotations_df['gene'] == 'CDKN1B']

We speculate that having access to this information during training is helping the model make better predictions on the effect of perturbing this gene.

<a id='#example_SAMD1+ZBTB1'></a>
### Two-gene Perturbation: Predicting perturbation response for perturbing the gene combination FOXA1+FOXL2

We can also look at effects of combination of genes. Below, we offer an example of predicting perturbation responses for the FOXA1+FOXL2 gene combination. <br>
Note that you can experiment with different gene combinations in **combo_seen0, combo_seen1**, or **combo_seen2**

In [ ]:
pert = 'FOXA1+FOXL2'
colors = ['fuchsia', 'green']
marker_types = [ 'o', 's', 's']
models_to_compare = ['scgpt', 'scgenept_ncbi+uniprot_gpt']

for model, color, marker_type, title  in zip(models_to_compare, colors, marker_types, ['scGPT', 'scGenePT_NCBI+UniProt']):
    plot_perturbation(trained_models[model], pert, model, color, marker_type, title, amp = True, pool_size=300)

Similarly, we can look at the NCBI and UniProt annotations for the genes:

In [ ]:
# NCBI Gene Card Annotations
NCBI_gene_card_summaries = json.load(open(gene_annotations_dir + 'NCBI_summary_of_genes.json', 'rb'))
# NCBI Gene Card + UniProt protein summaries Annotations
NCBI_UniProt_gene_card_protein_summaries = json.load(open(gene_annotations_dir + 'NCBI_UniProt_summary_of_genes.json', 'rb'))

comb_genes = pert.split('+')
for gene in comb_genes:
    print(f"Annotations for GENE: {gene}")
    print('='*30)
    print(f"NCBI Gene Card Summary: {NCBI_gene_card_summaries[gene]}")
    print('\n')
    print(f"NCBI Gene Card + UniProt Protein Summary: {NCBI_UniProt_gene_card_protein_summaries[gene]}")
    print('\n')

<a id='predicting_on_rnd_ctrl'></a>
## 3. Predicting on numpy arrays holding control samples

We can make predictions on various data formats. Let's say we have a control sample held in a numpy array. We can use one of the trained models to make predictions. <br> 
For example, let's use the scGenePT-GO-C model trained on Norman. First, we need to load the trained model: 

In [ ]:
model_name = 'scgenept_go_c_gpt_concat'
dataset_name = 'norman' 
pert_data = load_dataloader(dataset_name, batch_size = 64, val_batch_size = 64, split = 'simulation')

model_filename = model_name2model_variation[model_name]
model_location =  f'../models/finetuned/scgenept_go_c/{dataset_name}/{model_filename}'
model, gene_ids =  load_trained_scgenept_model(pert_data, model_name, pretrained_scgpt_model_dir, model_location, device, verbose = False)

Let's say we have a control sample in a np array and we want to predict the effect of perturbing the **FOSB** gene. Note that in the example below, the order of the genes in the ctrl_sample would have to match the one in gene_names of the dataset that the model was trained on. Otherwise, additional gene name matching should be made.

In [ ]:
gene_names = pert_data.gene_names.to_list()
print(f'There are {len(gene_names)} genes, the first 10 are: {gene_names[:10]}')

In [ ]:
pert = "FOSB+ctrl"
ctrl_sample = np.random.rand(5045) # this can be replaced with a different control sample, as long as the order of the gene matches

Then we need to assign the correct perturbation flags in the control sequence, so that the model knows what gene should get perturbed

In [ ]:
def get_pert_flags(ctrl_sample, gene_names, pert):
    pert_flags = np.zeros(len(ctrl_sample))
    if pert!= 'ctrl':
        for x in pert.split('+'):
            if x != 'ctrl':
                pert_flags[gene_names.index(x)] = 1
    pert_flags = torch.from_numpy(pert_flags).long().to(device).unsqueeze(0)
    return pert_flags

pert_flags = get_pert_flags(ctrl_sample, gene_names, pert)

Then we can get the model predictions by calling the model in inference mode:

In [ ]:
def pred_ctrl_sample(model, gene_ids, ctrl_sample, pert_flags, device):
    ctrl_sample = torch.from_numpy(np.expand_dims(ctrl_sample, 0)).to(dtype = torch.float32).to(device)
    gene_ids_tensor = torch.tensor(gene_ids).long().unsqueeze(0).to(device)
    src_key_padding_mask = torch.zeros_like(gene_ids_tensor, dtype=torch.bool, device=device)
    model = model.to(torch.float32)
    with torch.cuda.amp.autocast(enabled=True):
        with torch.no_grad():
            output_dict = model(
                gene_ids_tensor,
                ctrl_sample,
                pert_flags,
                src_key_padding_mask=src_key_padding_mask,
                CLS=False,
                CCE=False,
                MVC=False,
                ECS=False,
                do_sample=True,
            )
    prediction = output_dict["mlm_output"].float().detach().cpu().numpy()
    return prediction

In [ ]:
preds = pred_ctrl_sample(model, gene_ids, ctrl_sample, pert_flags, device)

In [ ]:
preds

In [ ]:
preds.shape

<a id='prediction_on_new_anndata_file'></a>
## 4. Predicting on AnnData files
We can follow a similar sequence to predict on an `AnnData` file. Note that in the example below, you can replace the `adata_ctrl` file with your file of choice:

In [ ]:
pert_adata = pert_data.adata
adata_ctrl = pert_adata[pert_adata.obs['condition'] == 'ctrl'][:100] # this can be any other AnnData file
adata_ctrl

Note that the set and order of the genes in the AnnData file would have to match that of the genes the model has been trained on. In most cases, this will mean filtering the data to match this. The 5045 genes used by the model are:

In [ ]:
adata_ctrl.var

Each of them will correspond to a value in `adata_ctrl.X`:

In [ ]:
adata_ctrl.X # an entry (i, j) corresponds to the value of gene j in cell i; the 5045 genes must match to the list above

Using similarly the model loaded above to generate predictions for perturbing a particular gene perturbation X:

In [ ]:
gene_pert = 'CEBPB+ctrl'

ctrl_size = None # Note that if this is != None, then the ctrl samples will get randomly sampled.
return_mean = False # If this is True, then the mean of the predictions will get returned
preds = model.pred_perturb_from_ctrl(adata_ctrl, gene_pert, gene_names, device, gene_ids, pool_size = ctrl_size, return_mean = False).squeeze()

In [ ]:
preds

In [ ]:
preds.shape

We can then assign the predictions to the anndata file:

In [ ]:
adata_ctrl.layers[f'{model_name}_predictions'] = preds

In [ ]:
adata_ctrl

<a id='contact'></a>
## Contact & Feedback

Ana-Maria Istrate, aistrate@chanzuckerberg.com

## References

1. Norman, Thomas M., et al. "Exploring genetic interaction manifolds constructed from rich single-cell phenotypes." Science 365.6455 (2019): 786-793.
2. Cui, Haotian, et al. "scGPT: toward building a foundation model for single-cell multi-omics using generative AI." Nature Methods (2024): 1-11.
3. Chen, Yiqun, and James Zou. "GenePT: a simple but effective foundation model for genes and cells built from ChatGPT." bioRxiv (2024): 2023-10.
4. Roohani, Yusuf, Kexin Huang, and Jure Leskovec. "Predicting transcriptional outcomes of novel multigene perturbations with GEARS." Nature Biotechnology 42.6 (2024): 927-935.